## CS336 - Lec3: Architectures, hyperparameters

## Lec3
### 学习内容
1. 标准Transformer
2. LLMs的共用点
3. Transformer结构和训练过程的常用变体

### 模型架构变体
#### Pre-Norm和Post-Norm
- 初始Transformer是在多头注意力MHA、FFN后加上的add&LN
- 现今几乎所有LM都将LN移到了MHA、FFN之前
- 好处：
    1. 不用warm-up，梯度一直存在且稳定
    2. 梯度传播不经过LN，整体的恒等映射让梯度传播更顺畅
    3. Pre-norm出现更少的梯度尖峰
- 现在还有在MHA、FFN后，残差连接前再加入一个LN的模型: Grok, Gemma 2


#### LayerNorm和RMSNorm
- LayerNorm: 归一化$d_{model}$维度上的均值和方差，$$y=\frac{x-\text{E}(x)}{\sqrt{\text{Var}(x)+\epsilon}}*\gamma+\beta$$

- RMSNorm: $$y=\frac{x}{\sqrt{||x||^2_2+\epsilon}}*\gamma$$
    - 效果相当，但是速度更快：不需要均值的计算，且不储存偏置
    - 在Transformer运行中，Norm的计算/计算消耗虽然只占0.17%(矩阵乘法类占99.8%)，但是占25.5%的运行时间
        - 因为数据储存的移动data movement占比大（内存访问效率）

受到RMSNorm启发，在FFN中也取消了偏置项，一个实证解释是减少内存且优化了稳定性。

#### 激活函数的选择（通常用于FFN部分）
ReLU -> GeLU -> GLU -> GeGLU -> *SwiGLU(最常用)

1. ReLU: $ReLU(x) = max(0, x)$
2. GeLU(使用高斯分布的累积密度函数$\Phi(x)$): $GELU(x) = x\Phi(x)$
3. Swish: $Swish(x) = x\times\text{sigmoid}(\beta x)$
    - $\beta$可以固定为1，也可以是训练参数，趋近正无穷收敛为ReLU,趋近0退化为线性

Gated activations(门控线性单元GLU)
- $(xW_1)\odot\sigma(xV)$，(xV)经过sigmoid值域落在(0,1)对前面的数据进行缩放
- 工程可选用：把输入向量前一半作为信号，后一半作为门控

常用GLU变体：
1. ReGLU: $ReGLU(x) = max(0, xW_1)\odot \sigma(xV)$
2. GeGLU: $GeGLU(x) = GELU(xW_1)\odot \sigma(xV)$
3. SwiGLU: $SwiGLU(x) = Swish(xW_1)\odot \sigma(xV)$
    - 工程中，如果对于W和V输入都为x而不砍维度，为了让SwiGLU-FFN的参数/计算量对齐原来的RELU-FFN，因此一般会让中间层的维度$=\frac83*d_{model}$（向上256取整）

#### 串行和并行 Serial/Parallel Layers

有一部分模型选择用并行Layers，像GPT-J、PaLM等
- 串行：$y=x+MLP(LN(x+Attention(LN(x))))$
- 并行：$y=x + MLP(LN(x)) + Attention(LN(x))$，对于参数更大的模型，提高15%速度但是质量几乎不变

#### 位置编码

1. 绝对位置编码: Embed(x,i)=v_x + u_i
    1. 三角位置嵌入: Embed(x,i)=v_x + PE(x,i)
        - $PE_{pos,2dim}=\sin(\frac{pos}{10000^{\frac{2dim}{d_{model}}}})$ and $PE_{pos,2dim+1}=\cos(\frac{pos}{10000^{\frac{2dim}{d_{model}}}})$
    2. BERT的可学习位置嵌入
    - 绝对位置编码的局限：变长序列推理
2. 相对位置编码
    1. RPE，修改自注意力机制（每次调用注意力都加入而不是直接在输入词向量时加入，只应用到Q和K上）的计算: $$e_{ij} = \frac{x_i W^Q (x_j W^K + \mathbf{a_{ij}^K})^T}{\sqrt{d_z}}$$ 其中 $\mathbf{a_{ij}^K}$ 是学习到的相对位置嵌入，代表位置 $i$ 和位置 $j$ 之间的距离。
        - 额外还会对相对位置i-j进行截断到k，即注意力最远看到的举例。
    2. 


旋转位置编码 RoPE, Rotary position embedding
- 核心理念：两个不同位置的编码的内积是一个以两个原向量和位置差值的函数，即$$<f(x,i)f(y,j)>=g(x,y,i-j)$$，也就是说注意力函数对位置的计算只会取决于相对位置i-j
- 实现：将d维向量看成d/2个2维平面，每个平面（两个维度）上利用线性代数的2D旋转矩阵对坐标进行旋转（m=pos,即在多少位置进行多少个theta的旋转），$$\begin{pmatrix} x'_1 \\ x'_2 \end{pmatrix} = 
\begin{pmatrix} \cos(m\theta) & -\sin(m\theta) \\ \sin(m\theta) & \cos(m\theta) \end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \end{pmatrix}$$
- 不同的维度，可以使用不同的旋转角度，相当于输入做矩阵乘法$$R_{\Theta, m}^d = \begin{pmatrix} 
\cos m\theta_1 & -\sin m\theta_1 & 0 & 0 & \cdots & 0 & 0 \\ 
\sin m\theta_1 & \cos m\theta_1 & 0 & 0 & \cdots & 0 & 0 \\ 
0 & 0 & \cos m\theta_2 & -\sin m\theta_2 & \cdots & 0 & 0 \\ 
0 & 0 & \sin m\theta_2 & \cos m\theta_2 & \cdots & 0 & 0 \\ 
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\ 
0 & 0 & 0 & 0 & \cdots & \cos m\theta_{d/2} & -\sin m\theta_{d/2} \\ 
0 & 0 & 0 & 0 & \cdots & \sin m\theta_{d/2} & \cos m\theta_{d/2} 
\end{pmatrix}$$

#### 超参数选择
1. FFN的维度比 ddf/d_model,对于ReLU激活一般是4，对于GLU类一般是8/3.
2. 多头注意力的head_dims*num_heads = d_model
3. Aspect ratio = d_model/n_layer，表示模型宽度深度，受并行结构影响基本保持在128左右

#### 词表大小
对于单语言模型，token数在30-50k，而对于多语言token数在100-250k

#### Dropout和正则化
现在的LLMs在预训练阶段数据>参数，一般只训练一个epoch，所以不考虑过拟合。
- 现在很少考虑用dropout
- 权重衰减 weight decay仍然有效，但不控制过拟合，与学习率schedule相关

### 稳定性技巧
Transformer中的softmax很容易出现数值不稳定，$e^{z_i}$可能很大

1. Output的softmax
在损失中加入对softmax分母项的惩罚，$+\alpha\log^2(Z(x))$

2. Attention的softmax
在计算出Q和K后，对其再进行一次LN；或者对logit进行截断（效果不佳）

### 注意力头变体
先回顾注意力机制里的计算，
- Compute = $bnd^2$ (+ $bn^2$，但这里是element-wise计算所以量级小可忽略)
- Memory Access = $bnd$(输入和输出) + $bhn^2$(所有注意力头都要产生一个中间矩阵) + $d^2$(4种权重向量)
- Arithmetic Intensity = 计算量/访存量
    - 这里可以计算出算数强度 $>O((\frac1d+\frac1{bn})^{-1})$
    - 这里值越大，表示GPU每一轮读取数据后，能进行更多的计算，减少了等数据搬运的时间。
    - 注意力机制的计算瓶颈在：中间矩阵。

KV cache
- 在推理过程中，我们没有大块的矩阵乘法，不能用并行，而是用自回归的方法逐次计算token
- 过程：
    1. 每次有一个新生成的token，生成q, k, v
    2. 从cache中提出已有key和value，和新生成的k，v拼接，与新的token(q)做attention，即可得到最终的输出
- 对于推理阶段的算数强度=$(bnd^2)/(bn^2d+nd^2)$ = O($(\frac nd+\frac1b)^{-1}$)
    - kv cache是一种类似于空间换时间的方法
    - batch越大，d_model越大，seq_len越小，则算术强度越大

- 问题：
    - query是新生成的词，那key，value怎么用的？
    - 为什么每次不用完整的query做attention：
        - 因为是自回归生成的token，所以计算attention时需要casaul mask，因此正好满足推理kv cache的需求
    - prefill（处理prompt）和decoding阶段

#### 多查询注意力 MQA/ 分组查询注意力 GQA
- 想法：基于kv cache，通过保持query的多头，减少k和v到单头实现
- 结果：更少的cache进出内存
- 总共的内存access变为$(bnd+bn^2k+nd^2)$，算数强度=O($(\frac 1d +\frac n{dh}+\frac1b)^{-1}$)
- GQA：控制key-query ration，从而增加一定的kv头
- MQA伤害性能，而GQA几乎不影响

#### Sparse or sliding window attention
- 一般来说q和k计算因为自回归只会算出nxn矩阵的上三角矩阵，现在对下三角矩阵计算出的数值进行调整，比如只用最近的3个token的计算结果（类似于滑动窗口），因此下三角矩阵是稀疏的

### 其他尝试的trick
在Cohere Command A模型中，使用的3层普通Attention层+1层不加位置编码的Full Attention层，这样的结构会让模型进行更激进的外推

### 